# Mean Teacher Bert Assessment



# Requirement Installation 

In [2]:
#installing requirements
!pip install textattack==0.3.0

     |████████████████████████████████| 359 kB 7.8 MB/s 
     |████████████████████████████████| 264 kB 62.7 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
     |████████████████████████████████| 284 kB 60.1 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 2.6 MB 64.1 MB/s 
     |████████████████████████████████| 769 kB 56.3 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 636 kB 66.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.3 MB/s 
     |████████████████████████████████| 243 kB 63.2 MB/s 
     |████████████████████████████████| 118 kB 79.4 MB/s 
     |████████████████████████████████| 69 kB 10.4 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
     |██████████████

# Importing and Declaration

In [3]:
import os 
os.chdir('/content/drive/MyDrive/Master Thesis/Mean-Teacher-BERT-assessment/')

In [4]:
# importing libraries 
import os 
import pandas as pd 
import numpy as np
import tensorflow as tf

from ipywidgets import *
from IPython.display import display


from Utils.utils import label_to_int, dataset_split, convert_to_category, create_tokenizer, data_tokenization
from Utils.data_loader import data_loader
from Bert.bert_model import bert_model
from Attack_model.TextAttack import attack_model


from MeanTeacher.train_mean_teacher import train_mean_teacher

import random 
random.seed(42)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 40.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmppnkd557_.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


# Hyperparameter selections

In [5]:
# Parameter declaration 
max_len=widgets.BoundedIntText(value=100,min=10,max=512,step=10,description='Max Length:',disabled=False,)

learning_rate= widgets.Dropdown(options=[2e-5, 1e-5, 1e-3],value=2e-5,description='Learning Rate:',disabled=False,)

epochs= widgets.Dropdown(options=[3, 4, 5,6],value=3,description='Epochs:',disabled=False,)

batch_size= widgets.Dropdown(options=[4, 5,6,7,8],value=4,description='Batch Size:',disabled=False,)

#for mean teacher model
alpha= widgets.BoundedFloatText(value=0.999,min=0.90,max=0.999,step=0.001,description='Alpha:',disabled=False, readout_format='.3f')
ratio= widgets.BoundedFloatText(value=0.5,min=0.1,max=0.9,step=0.1,description='Ratio:',disabled=False, readout_format='.2f')

tf.print('\n Max Length: Length of the article')
display(max_len)
tf.print('\n Learning rate: Learning rate for training the model')
display(learning_rate)
tf.print('\n Epochs')
display(epochs)
tf.print('\n batch_size')
display(batch_size)
tf.print('\n Alpha: Parameter related to Mean teacher model, its the proportion of student weight that teacher model will get each step.')
display(alpha)
tf.print("\n Ratio: Parameter related to Mean Teacher model, that manage the proportion of classification cost and consistency cost in calculating overall cost.")
display(ratio)
# widgets.HBox([max_len, learning_rate, epochs, batch_size,alpha,ratio])


 Max Length: Length of the article


BoundedIntText(value=100, description='Max Length:', max=512, min=10, step=10)


 Learning rate: Learning rate for training the model


Dropdown(description='Learning Rate:', options=(2e-05, 1e-05, 0.001), value=2e-05)


 Epochs


...

Dropdown(description='Batch Size:', options=(4, 5, 6, 7, 8), value=4)


 Alpha: Parameter related to Mean teacher model, its the proportion of student weight that teacher model will get each step.


BoundedFloatText(value=0.999, description='Alpha:', max=0.999, min=0.9, step=0.001)


 Ratio: Parameter related to Mean Teacher model, that manage the proportion of classification cost and consistency cost in calculating overall cost.


BoundedFloatText(value=0.5, description='Ratio:', max=0.9, min=0.1, step=0.1)

In [6]:
# Details related to the dataset
# columns names 
feature_col=widgets.Text(value='tweet',placeholder='Enter Feature Col Name',description='Feature Column:',disabled=False)
target_col=widgets.Text(value='label',placeholder='Enter Target Col Name',description='Target Column:',disabled=False) 
aug_col=widgets.Text(value='aug_tweet',placeholder='Enter Augmented data Col Name',description='Augment Column:',disabled=False)   

#pretrained model details
pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,) 

widgets.HBox([feature_col,target_col,aug_col,pretrained_weights])

In [7]:
#Data location 
Data_loc=widgets.Text(value='Data/codalab/',placeholder='Enter Data location ',description=' Data Location Column:',disabled=False)   
widgets.HBox([Data_loc])

# Reading dataset

In [13]:
df_train,df_test,df_aug_unlabel= data_loader(Data_loc.value,target_col.value,aug_col.value)

# Training 

<font color='blue'>This section is to train and save the model. <br>
__If model is already trained and saved, directly move forward for attack and evaluation section.__ </font>

## BERT Model Training

In [ ]:
pretrained_weights.value

'bert-base-uncased'

In [ ]:
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:
bert=bert_model(pretrained_weights.value,max_len.value,learning_rate.value)
bert.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem_5 (Sli (None, 768)          0           tf_bert_model_2[0][0]      

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size.value,epochs=epochs.value,verbose=1)

Epoch 1/3


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Gradients do not exist for variables ['tf_bert_model_2/bert/pooler/dense/kernel:0', 'tf_bert_model_2/bert/pooler/dense/bias:0'] when minimizing the loss.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Gradients do not exist for variables ['tf_bert_model_2/bert/pooler/dense/kernel:0', 'tf_bert_model_2/bert/pooler/dense/bias:0'] when minimizing the loss.


1523/1523 [==============================] - 408s 255ms/step - loss: 0.2502 - accuracy: 0.8975
Epoch 2/3
1523/1523 [==============================] - 387s 254ms/step - loss: 0.0839 - accuracy: 0.9698
Epoch 3/3
1523/1523 [==============================] - 388s 254ms/step - loss: 0.0312 - accuracy: 0.9908


In [ ]:
#saving the model 
if  pretrained_weights.value == 'distilbert-base-uncased':
    bert.save_weights('SavedModels/Codalab/DistillBERT_model.h5')
else:
    bert.save_weights('SavedModels/Codalab/BERT_model.h5')

## Adversarial Mean Teacher BERT Training

In [ ]:
# Training
tokenizer=create_tokenizer(pretrained_weights.value)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col.value,target_col.value,max_len.value,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col.value,target_col.value,max_len.value,tokenizer)
x_unlabel,_,_=data_tokenization(df_aug_unlabel,aug_col.value,target_col.value,max_len.value,tokenizer)

In [ ]:

with tf.device('/GPU:0'):
    student, teacher= train_mean_teacher(x_train, y_train, x_unlabel, 
                                         pretrained_weights.value,
                                         epochs.value,batch_size.value,
                                         learning_rate.value,max_len.value,alpha.value)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without 

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


alpha: 0.999

epoch 1
1522/1523 [============================>.] - ETA: 1s - Accuracy: 0.6991 - Overall_Loss: 0.0982
epoch 2
1522/1523 [============================>.] - ETA: 1s - Accuracy: 0.8422 - Overall_Loss: 0.0223
epoch 3
1522/1523 [============================>.] - ETA: 1s - Accuracy: 0.8946 - Overall_Loss: 0.0065

In [ ]:
if  pretrained_weights.value == 'distilbert-base-uncased':
    teacher.save_weights('SavedModels/Codalab/DistillBERT_teacher.h5')
else:
    teacher.save_weights('SavedModels/Codalab/BERT_teacher.h5')


# Attack And Evaluation

In [8]:
import textattack

In [11]:
# bert_to_attack=widgets.Textarea(value='SavedModels/Codalab/BERT_model.h5',placeholder='Enter Model location ',description=' BERT Model location:',disabled=False)
# MTbert_to_attack=widgets.Textarea(value='SavedModels/Codalab/BERT_teacher.h5',placeholder='Enter Model location ',description=' MTBERT Model location:',disabled=False)   
# pretrained_weights=widgets.Dropdown(options=['bert-base-uncased','distilbert-base-uncased' ],value='bert-base-uncased',description='Pretrained Model:',disabled=False,) 
model_name=widgets.Dropdown(options=['BERT','DistillBERT']
                               ,value='BERT',
                               description='Model Name:',
                               disabled=False,) 
tf.print('\n model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter')
display(model_name)
if model_name=='BERT':
  bert_to_attack= 'SavedModels/Codalab/BERT_model.h5'
  MTbert_to_attack='SavedModels/Codalab/BERT_teacher.h5'
  pretrained_weights.value = 'bert-base-uncased'
else:
  bert_to_attack= 'SavedModels/Codalab/DistillBERT_model.h5'
  MTbert_to_attack='SavedModels/Codalab/DistillBERT_teacher.h5'
  pretrained_weights.value = 'distilbert-base-uncased'


# tf.print('\n Please select the location of BERT and MT BERT model.\n')
# display(bert_to_attack, MTbert_to_attack)
# tf.print('\n Please select the pretrained model weights.')
# display(pretrained_weights)



 model_name: Model name for attack.Note : Please confirm with the bert_to_attack parameter


Dropdown(description='Model Name:', options=('BERT', 'DistillBERT'), value='BERT')

In [15]:
print(model_name.value)

DistillBERT


In [16]:
# attack parameters 

# Query Budget is maximum query allowed
query_budget= widgets.BoundedIntText(value=200,min=10,max=500,step=20,description='Query Budget:',disabled=False,)

# number of example to take for attack from test dataset randomly 
num_examples=widgets.BoundedIntText(value=200,min=10,max=len(df_test),step=20,description='Number of Example:',disabled=False,)

# attack recipes 
attack_recipe=widgets.Dropdown(options=['TextFoolerJin2019','PWWSRen2019','BERTAttackLi2020','BAEGarg2019','TextBuggerLi2018']
                               ,value='TextFoolerJin2019',
                               description='Attack Recipe:',
                               disabled=False,) 


tf.print('\n query_budget: Maximum Query allowed to attack model')
display(query_budget)
tf.print('\n num_examples: Number of example to take for attack from test dataset randomly. ')
display(num_examples)
tf.print('\n attack_recipe: Attack methodolgy to attack the model. ')
display(attack_recipe)



 query_budget: Maximum Query allowed to attack model


BoundedIntText(value=200, description='Query Budget:', max=500, min=10, step=20)


 num_examples: Number of example to take for attack from test dataset randomly. 


BoundedIntText(value=200, description='Number of Example:', max=2040, min=10, step=20)


 attack_recipe: Attack methodolgy to attack the model. 


Dropdown(description='Attack Recipe:', options=('TextFoolerJin2019', 'PWWSRen2019', 'BERTAttackLi2020', 'BAEGa…

- __PWWSRen 2019 - Generating Natural Language Adversarial Examples through Probability Weighted Word Saliency. Shuhuai Ren, Yihe Deng, Kun He, Wanxiang Che__<br>
Based on the synonyms substitution strategy, they introduced a new word replacement order determined by both the word saliency and the classification probability, and propose a greedy algorithm called probability weighted word saliency (PWWS) for text adversarial attack.
- __TextFoolerJin2019 - Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment.__<br>
In this paper, we present TextFooler, a simple but strong baseline to generate natural adversarial text. By applying it to two fundamental natural language tasks, text classification and textual entailment, we successfully attacked three target models, including the powerful pre-trained BERT, and the widely used convolutional and recurrent neural networks. We demonstrate the advantages of this framework in three ways: (1) effective---it outperforms state-of-the-art attacks in terms of success rate and perturbation rate, (2) utility-preserving---it preserves semantic content and grammaticality, and remains correctly classified by humans, and (3) efficient---it generates adversarial text with computational complexity linear to the text length. *The code, pre-trained target models, and test examples are available at https://github.com/jind11/TextFooler.
- __BAEGarg2019-BAE (BAE: BERT-Based Adversarial Examples)__ <br>
BAE, a black box attack for generating adversarial examples using contextual perturbations from a BERT masked language model. BAE replaces and inserts tokens in the original text by masking a portion of the text and leveraging the BERT-MLM to generate alternatives for the masked tokens. Through automatic and human evaluations, we show that BAE performs a stronger attack, in addition to generating adversarial examples with improved grammaticality and semantic coherence as compared to prior work.
- __BERTAttackLi2020 - BERT-ATTACK: Adversarial Attack Against BERT Using BERT.__ <br>
we propose \textbf{BERT-Attack}, a high-quality and effective method to generate adversarial samples using pre-trained masked language models exemplified by BERT. We turn BERT against its fine-tuned models and other deep neural models for downstream tasks. Our method successfully misleads the target models to predict incorrectly, outperforming state-of-the-art attack strategies in both success rate and perturb percentage, while the generated adversarial samples are fluent and semantically preserved. Also, the cost of calculation is low, thus possible for large-scale generations.
- __TextBugger: Generating Adversarial Text Against Real-world Applications.__ <br>
Deep Learning-based Text Understanding (DLTU) is the backbone technique behind various applications, including question answering, machine translation, and text classification. Despite its tremendous popularity, the security vulnerabilities of DLTU are still largely unknown, which is highly concerning given its increasing use in security-sensitive applications such as sentiment analysis and toxic content detection. In this paper, we show that DLTU is inherently vulnerable to adversarial text attacks, in which maliciously crafted texts trigger target DLTU systems and services to misbehave. Specifically, we present TextBugger, a general attack framework for generating adversarial texts. In contrast to prior works, TextBugger differs in significant ways: (i) effective -- it outperforms state-of-the-art attacks in terms of attack success rate; (ii) evasive -- it preserves the utility of benign text, with 94.9\% of the adversarial text correctly recognized by human readers; and (iii) efficient -- it generates adversarial text with computational complexity sub-linear to the text length. We empirically evaluate TextBugger on a set of real-world DLTU systems and services used for sentiment analysis and toxic content detection, demonstrating its effectiveness, evasiveness, and efficiency. For instance, TextBugger achieves 100\% success rate on the IMDB dataset based on Amazon AWS Comprehend within 4.61 seconds and preserves 97\% semantic similarity. We further discuss possible defense mechanisms to mitigate such attack and the adversary's potential countermeasures, which leads to promising directions for further research.


In [27]:
'''
Some of the attack recipe provided by textattack is not working with lengthy article. So, 
I am filtering the with the length. May be in future I will remove it.
'''
df_test['len']=df_test.apply(lambda row: len(row[feature_col.value].split()),axis=1)
df_test_for_attack= df_test[(df_test['len']>0) & (df_test['len']<20)][[feature_col.value,target_col.value]].dropna().reset_index(drop=True)
df_test_for_attack=df_test_for_attack.sample(frac=1).reset_index(drop=True)
print('Length of dataset', len(df_test_for_attack))


#creating dataset for the attacking
dataset_for_attack=list(df_test_for_attack.itertuples(index=False, name=None))
dataset_for_attack = textattack.datasets.Dataset(dataset_for_attack)


Length of dataset 790


### BERT model

In [28]:
bert= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert.load_weights(bert_to_attack)

attack_model(bert, 
             dataset_for_attack,
             pretrained_weights.value,
             max_len.value,
             model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value )

Model selected: distilbert-base-uncased


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/B

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.4
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



...


[Succeeded / Failed / Skipped / Total] 9 / 0 / 2 / 11:   2%|▏         | 12/500 [01:01<42:00,  5.17s/it]
[Succeeded / Failed / Skipped / Total] 10 / 0 / 2 / 12:   2%|▏         | 12/500 [01:02<42:01,  5.17s/it]

--------------------------------------------- Result 12 ---------------------------------------------
1 (73%) --> 0 (68%)

 contact tracing apps have been secretly installed on every android phone  

 contact trace apps have been since installed on every android phone  





[Succeeded / Failed / Skipped / Total] 10 / 0 / 2 / 12:   3%|▎         | 13/500 [01:49<1:08:04,  8.39s/it]
[Succeeded / Failed / Skipped / Total] 11 / 0 / 2 / 13:   3%|▎         | 13/500 [01:49<1:08:05,  8.39s/it]

--------------------------------------------- Result 13 ---------------------------------------------
1 (73%) --> 0 (55%)

trump s covid test came back positive for stupidity donaldtrump whitehouse coronavirus c

trump s cox test came back positive for stupidity donaldtrumthe whitehouse coronavirus c




KeyboardInterrupt: ignored

### MTBERT model

In [ ]:
bert_teacher= bert_model(pretrained_weights.value,max_len.value,learning_rate.value) 
bert_teacher.load_weights(MTbert_to_attack)

attack_model(bert_teacher, 
             dataset_for_attack,
             pretrained_weights.value,
             max_len.value,
             'MT'+model_name.value,
             attack_recipe=attack_recipe.value,
             query_budget= query_budget.value,
             num_examples=num_examples.value )